In [1]:
from __future__ import print_function
from future.utils import iteritems
import json
import numpy as np
import os
import argparse
import sys
from sklearn.metrics import confusion_matrix

import mir_eval

In [12]:
f = open('./evaluation/filelist.txt')
filelist = f.readlines()
f.close()
estDir = '/Users/chrismiller/Desktop/Work/NYU-MIR/tmc-ace/matlab/results/'

In [14]:
# counter for mirex_scores array
counter = 0;
mirex_scores = []

# loop through tracks
for track in filelist:
    # parse filelist, finding reference annotations and estimated results
    track = track.strip()
    (refDir, refFile) = os.path.split(track)
    (name, ext) = os.path.splitext(refFile)
    referenceFilename = refDir + '/' + name + '.lab'
    estimatedFilename = estDir + name + '.wav.txt'
    ref_starts, ref_ends, ref_labels = mir_eval.io.load_delimited(referenceFilename, [float, float, str], r'\s+')
    est_starts, est_ends, est_labels = mir_eval.io.load_delimited(estimatedFilename, [float, float, str], r'\s+')
    # filter by chordType
    filtered_ref_starts = []
    filtered_ref_ends = []
    filtered_est_starts = []
    filtered_est_ends = []
    filtered_ref_labels = []
    filtered_est_labels = []
    for i in range(len(ref_labels)):
        currentChordQuality = mir_eval.chord.split(ref_labels[i])[1]
        if currentChordQuality == chordType:
            filtered_ref_starts.append(ref_starts[i])
            filtered_ref_ends.append(ref_ends[i])
            filtered_ref_labels.append(ref_labels[i])
    for i in range(len(est_labels)):
        currentChordQuality = mir_eval.chord.split(est_labels[i])[1]
        if currentChordQuality == chordType:
            filtered_est_starts.append(est_starts[i])
            filtered_est_ends.append(est_ends[i])
            filtered_est_labels.append(est_labels[i])
    # assemble interval labels from starts and ends
    filtered_ref_idx = np.array([filtered_ref_starts, filtered_ref_ends]).T
    filtered_est_idx = np.array([filtered_est_starts, filtered_est_ends]).T
    # Validate filtered intervals, and throw a warning in place of an error
    try:
        mir_eval.util.validate_intervals(filtered_ref_idx)
        mir_eval.util.validate_intervals(filtered_est_idx)
    except ValueError as error:
        warnings.warn(error.args[0]) 
    # compute scores and grab mirex for averaging
    scores = mir_eval.chord.evaluate(filtered_ref_idx, filtered_ref_labels, filtered_est_idx, filtered_est_labels)
    mirex_scores.append(scores['mirex'])
    counter += 1

In [16]:
track = filelist[0]
track = track.strip()
(refDir, refFile) = os.path.split(track)
(name, ext) = os.path.splitext(refFile)
referenceFilename = refDir + '/' + name + '.lab'
estimatedFilename = estDir + name + '.wav.txt'
# str(referenceFilename)
# str(estimatedFilename)

# (ref_idx, ref_labels) = mir_eval.io.load_labeled_intervals(referenceFilename)
# (est_idx, est_labels) = mir_eval.io.load_labeled_intervals(estimatedFilename)
ref_starts, ref_ends, ref_labels = mir_eval.io.load_delimited(referenceFilename, [float, float, str], r'\s+')
est_starts, est_ends, est_labels = mir_eval.io.load_delimited(estimatedFilename, [float, float, str], r'\s+')

In [13]:
est_labels

['N',
 'F:maj',
 'G:7',
 'G:sus4',
 'C:maj',
 'D:7',
 'C:maj',
 'F:sus2',
 'F:maj',
 'C:min',
 'G:min7',
 'C:7',
 'C:sus4',
 'F:sus4',
 'F:maj',
 'C:maj',
 'C:7',
 'A:min7',
 'C:sus4',
 'C:maj',
 'C:sus4',
 'C:maj',
 'C:min',
 'Bb:maj',
 'C:sus4',
 'C:maj',
 'C:sus4',
 'C:min',
 'N']

In [ ]:
# filter by chordType
filtered_ref_starts = []
filtered_ref_ends = []
filtered_est_starts = []
filtered_est_ends = []
filtered_ref_labels = []
filtered_est_labels = []

for i in range(len(ref_labels)):
    currentChordQuality = mir_eval.chord.split(ref_labels[i])[1]
    if currentChordQuality == chordType:
        filtered_ref_starts.append(ref_starts[i])
        filtered_ref_ends.append(ref_ends[i])
        filtered_ref_labels.append(ref_labels[i])

for i in range(len(est_labels)):
    currentChordQuality = mir_eval.chord.split(est_labels[i])[1]
    if currentChordQuality == chordType:
        filtered_est_starts.append(est_starts[i])
        filtered_est_ends.append(est_ends[i])
        filtered_est_labels.append(est_labels[i])

# assemble interval labels from starts and ends
filtered_ref_idx = np.array([filtered_ref_starts, filtered_ref_ends]).T
filtered_est_idx = np.array([filtered_est_starts, filtered_est_ends]).T

# Validate filtered intervals, and throw a warning in place of an error
try:
    mir_eval.util.validate_intervals(filtered_ref_idx)
    mir_eval.util.validate_intervals(filtered_est_idx)
except ValueError as error:
    warnings.warn(error.args[0])
        
# compute scores and grab mirex for averaging
scores = mir_eval.chord.evaluate(filtered_ref_idx, filtered_ref_labels, filtered_est_idx, filtered_est_labels)
mirex_scores[i] = scores['mirex']
i += 1

In [ ]:
e_int, e_lbl = mir_eval.util.adjust_intervals(filtered_est_idx, filtered_est_labels, filtered_ref_idx.min(), filtered_ref_idx.max(),NO_CHORD, NO_CHORD)

intervals, r_lbl, e_lbl = mir_eval.util.merge_labeled_intervals(filtered_ref_idx, filtered_ref_labels, filtered_est_idx, filtered_est_lables)

In [ ]:
# index for filtered chord scoring

# assemble interval labels from starts and ends for entire track
full_ref_idx = np.array([ref_starts, ref_ends]).T
full_est_idx = np.array([est_starts, est_ends]).T

e_int, e_lbl = mir_eval.util.adjust_intervals(full_est_idx, est_labels, full_ref_idx.min(), full_ref_idx.max(),'N', 'N')
intervals, r_lbl, e_lbl = mir_eval.util.merge_labeled_intervals(ref_idx, ref_labels, e_int, e_lbl)

# loop through reference arrays
for i in range(len(r_lbl)):
    currentChordQuality = mir_eval.chord.split(r_lbl[i])[1]
    if currentChordQuality == chordType:
        filtered_ref_starts.append(intervals[i,0])
        filtered_ref_ends.append(intervals[i,1])
        filtered_ref_labels.append(r_lbl[i])
        filtered_est_labels.append(e_lbl[i])
        i




In [ ]:
scores = {}
mirex = []
tetrads = []
root = []
for track in filelist:
        # parse filelist, finding reference annotations and estimated results
        track = track.strip()
        (refDir, refFile) = os.path.split(track)
        (name, ext) = os.path.splitext(refFile)
        referenceFilename = refDir + '/' + name + '.lab'
        estimatedFilename = estDir + name + '.wav.txt'
        # load labels in reference/estimated files
        (ref_idx, ref_labels) = mir_eval.io.load_labeled_intervals(referenceFilename)
        (est_idx, est_labels) = mir_eval.io.load_labeled_intervals(estimatedFilename)
        # compute scores 
        scores = mir_eval.chord.evaluate(ref_idx, ref_labels, est_idx, est_labels)
        mirex.append(scores['mirex'])
        tetrads.append(scores['tetrads'])
        root.append(scores['root'])

In [ ]:
# parse filelist, finding reference annotations and estimated results
track = track.strip()
(refDir, refFile) = os.path.split(track)
(name, ext) = os.path.splitext(refFile)
referenceFilename = refDir + '/' + name + '.lab'
estimatedFilename = estDir + name + ext + '.txt'

# load starts, ends, and labels in reference/estimated files
ref_starts, ref_ends, ref_labels = mir_eval.io.load_delimited(referenceFilename, [float, float, str], r'\s+')
est_starts, est_ends, est_labels = mir_eval.io.load_delimited(estimatedFilename, [float, float, str], r'\s+')
   
# build index arrays over whole track and sync to reference
full_ref_idx = np.array([ref_starts, ref_ends]).T
full_est_idx = np.array([est_starts, est_ends]).T

e_int, e_lbl = mir_eval.util.adjust_intervals(full_est_idx, est_labels, full_ref_idx.min(), full_ref_idx.max(),'N', 'N')
intervals, r_lbl, e_lbl = mir_eval.util.merge_labeled_intervals(full_ref_idx, ref_labels, e_int, e_lbl)


#########################################################
# filter by chordType

maj_ref_labels = []
maj_est_labels = []
min_ref_labels = []
min_est_labels = []

# loop through reference arrays
for i in range(len(r_lbl)):
    currentChordQuality = mir_eval.chord.split(r_lbl[i])[1]
    if currentChordQuality == 'maj':
        maj_ref_labels.append(mir_eval.chord.split(r_lbl[i])[1])
        maj_est_labels.append(mir_eval.chord.split(e_lbl[i])[1])
    if currentChordQuality == 'min':
        min_ref_labels.append(mir_eval.chord.split(r_lbl[i])[1])
        min_est_labels.append(mir_eval.chord.split(e_lbl[i])[1])
